# DrQA
This is a PyTorch implementation of the DrQA system described in the ACL 2017 paper [Reading Wikipedia to Answer Open-Domain Questions](https://arxiv.org/abs/1704.00051).



## Quick Links

- [About](#machine-reading-at-scale)
- [Demo](#quick-start-demo)
- [Installation](#installing-drqa)
- [Components](#drqa-components)


## Machine Reading at Scale

DrQA is a system for reading comprehension applied to open-domain question answering. In particular, DrQA is targeted at the task of "machine reading at scale" (MRS). In this setting, we are searching for an answer to a question in a potentially very large corpus of unstructured documents (that may not be redundant). Thus the system has to combine the challenges of document retrieval (finding the relevant documents) with that of machine comprehension of text (identifying the answers from those documents).

Our experiments with DrQA focus on answering factoid questions while using Wikipedia as the unique knowledge source for documents. Wikipedia is a well-suited source of large-scale, rich, detailed information. In order to answer any question, one must first retrieve the few potentially relevant articles among more than 5 million, and then scan them carefully to identify the answer.

Note that DrQA treats Wikipedia as a generic collection of articles and does not rely on its internal graph structure. As a result, **_DrQA can be straightforwardly applied to any collection of documents_**, as described in the retriever [README](scripts/retriever/README.md).

This repository includes code, data, and pre-trained models for processing and querying Wikipedia as described in the paper -- see [Trained Models and Data](#trained-models-and-data). We also list several different datasets for evaluation, see [QA Datasets](#qa-datasets). Note that this work is a refactored and more efficient version of the original code. Reproduction numbers are very similar but not exact.


## Quick Start: Demo on Wikipedia

[Install](#installing-drqa) DrQA and [download](#trained-models-and-data) models to start asking open-domain questions on Wikipedia!

Run `python scripts/pipeline/interactive.py` to drop into an interactive session. For each question, the top span and the Wikipedia paragraph it came from are returned.


```
>>> process('What is question answering?')

Top Predictions:
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+
| Rank |                                                  Answer                                                  |        Doc         | Answer Score | Doc Score |
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+
|  1   | a computer science discipline within the fields of information retrieval and natural language processing | Question answering |    1917.8    |   327.89  |
+------+----------------------------------------------------------------------------------------------------------+--------------------+--------------+-----------+

Contexts:
[ Doc = Question answering ]
Question Answering (QA) is a computer science discipline within the fields of
information retrieval and natural language processing (NLP), which is
concerned with building systems that automatically answer questions posed by
humans in a natural language.
```

```
>>> process('What is the answer to life, the universe, and everything?')

Top Predictions:
+------+--------+---------------------------------------------------+--------------+-----------+
| Rank | Answer |                        Doc                        | Answer Score | Doc Score |
+------+--------+---------------------------------------------------+--------------+-----------+
|  1   |   42   | Phrases from The Hitchhiker's Guide to the Galaxy |    47242     |   141.26  |
+------+--------+---------------------------------------------------+--------------+-----------+

Contexts:
[ Doc = Phrases from The Hitchhiker's Guide to the Galaxy ]
The number 42 and the phrase, "Life, the universe, and everything" have
attained cult status on the Internet. "Life, the universe, and everything" is
a common name for the off-topic section of an Internet forum and the phrase is
invoked in similar ways to mean "anything at all". Many chatbots, when asked
about the meaning of life, will answer "42". Several online calculators are
also programmed with the Question. Google Calculator will give the result to
"the answer to life the universe and everything" as 42, as will Wolfram's
Computational Knowledge Engine. Similarly, DuckDuckGo also gives the result of
"the answer to the ultimate question of life, the universe and everything" as
42. In the online community Second Life, there is a section on a sim called
43. "42nd Life." It is devoted to this concept in the book series, and several
attempts at recreating Milliways, the Restaurant at the End of the Universe, were made.
```

```
>>> process('Who was the winning pitcher in the 1956 World Series?')

Top Predictions:
+------+------------+------------------+--------------+-----------+
| Rank |   Answer   |       Doc        | Answer Score | Doc Score |
+------+------------+------------------+--------------+-----------+
|  1   | Don Larsen | New York Yankees |  4.5059e+06  |   278.06  |
+------+------------+------------------+--------------+-----------+

Contexts:
[ Doc = New York Yankees ]
In 1954, the Yankees won over 100 games, but the Indians took the pennant with
an AL record 111 wins; 1954 was famously referred to as "The Year the Yankees
Lost the Pennant". In , the Dodgers finally beat the Yankees in the World
Series, after five previous Series losses to them, but the Yankees came back
strong the next year. On October 8, 1956, in Game Five of the 1956 World
Series against the Dodgers, pitcher Don Larsen threw the only perfect game in
World Series history, which remains the only perfect game in postseason play
and was the only no-hitter of any kind to be pitched in postseason play until
Roy Halladay pitched a no-hitter on October 6, 2010.
```

## Installing DrQA
Please use a multiGPU big system as this is very memory intensive.


### Dependencies
DrQA requires Linux/OSX and Python 3.5 or higher. It also requires installing [PyTorch](http://pytorch.org/) (version 0.4.0 is not supported yet). Its other dependencies are listed in requirements.txt. CUDA is needed.



### Setup Environment
```
conda create --name <myenv>
source activate <myenv>
cd /home/<user>/notebooks
git clone https://github.com/antriv/DrQA.git
cd DrQA
pip install -r requirements.txt
python setup.py develop
```
To install pytorch:
```
# install torch (with cuda 9)
conda install pytorch torchvision cuda90 -c pytorch
# test gpu install
python -c 'import torch; print(torch.rand(2,3).cuda())'
```
**IMPORTANT: The default [tokenizer](#tokenizers) is CoreNLP so you will need that in your `CLASSPATH` to run the examples.**
If you do not already have a CoreNLP [download](https://stanfordnlp.github.io/CoreNLP/index.html#download) you can run:
```bash
./install_corenlp.sh
```
Verify that it runs:
```python
from drqa.tokenizers import CoreNLPTokenizer
tok = CoreNLPTokenizer()
tok.tokenize('hello world').words()  # Should complete immediately
```
For convenience, the Document Reader, Retriever, and Pipeline modules will try to load default models if no model argument is given. See below for downloading these models.



### Trained Models and Data

To download all provided trained models and data for Wikipedia question answering, run:

```bash
./download.sh
```

_Warning: this downloads a 7.5GB tarball (25GB untarred) and will take some time._

This stores the data in `data/` at the file paths specified in the various modules' defaults

## DrQA Components


### Document Reader
DrQA's Document Reader is a multi-layer recurrent neural network machine comprehension model trained to do extractive question answering. That is, the model tries to find the answer to any question as a text span in one of the returned documents.

The Document Reader was inspired by, and primarily trained on, the [SQuAD](https://arxiv.org/abs/1606.05250) dataset. It can also be used standalone on such SQuAD-like tasks where a specific context is supplied with the question, the answer to which is contained in the context. Here we use the pretrained on already downloaded.


### Document Retriever
DrQA is not tied to any specific type of retrieval system -- as long as it effectively narrows the search space and focuses on relevant documents.
To efficiently store and access our documents, we store them in a sqlite database. The key is the `doc_id` and the value is the `text`.


### Building the TF-IDF N-grams
For efficient Document retrieval, we need to build a TF-IDF weighted word-doc sparse matrix from the documents stored in the sqlite db.


### SqlLite DB & TF-IDF for Gutenberg data
To test Gutenberg data, we created a sqlite db and TF-IDF from ther Gutenberg corpus of documents. Download preprocessed data from https://1drv.ms/f/s!Au1eBA3u53Uh1HA0v8ukhhYSVMuC. Download the data and place the downloaded 'gutenberg' folder in the main 'DrQA' folder.


### DrQA Pipeline
The full system is linked together in `drqa.pipeline.DrQA`.
To interactively ask questions using the full DrQA:

In [ ]:
!python scripts/pipeline/interactive.py --retriever-model '<path to TF-IDF>/gutenbergchildrendb-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz' --doc-db '<path to DB>/gutenbergchildrendb.db'

#### To test the system, below are some sample query: 

process('What did the little pig do when the wolf attacked?')

process('Who is mad hatter?', candidates=None, top_n=1, n_docs=5)